To start the merge, we install the rapidfuzz library if it will load the code faster to see the coincidence threshold, we will do the merge based on the coincidences of the album, artist and track name parameters, which will match them according to their level of coincidence with the other dataset and we will print the result every 100 rows.

In [ ]:
import os
import pandas as pd
import re
from rapidfuzz import process, fuzz

# Función para normalizar texto
def normalize_text(text):
    if isinstance(text, str):
        text = text.lower()
        text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # Eliminar caracteres especiales
        text = re.sub(r'\s+', ' ', text).strip()  # Eliminar espacios adicionales
        text = re.sub(r'\b(feat|ft|featuring|remix|version)\b', '', text)  # Eliminar términos problemáticos
    return text

# Cargar los archivos CSV
spotify_csv_path = r'C:\Users\USUARIO\Documents\Workshop2\data\processed\spotify_cleaned.csv'
grammys_csv_path = r'C:\Users\USUARIO\Documents\Workshop2\data\processed\grammys_supercleaned.csv'  # Usamos el archivo con spotify_id

spotify_merge = pd.read_csv(spotify_csv_path)
grammy_merge = pd.read_csv(grammys_csv_path)

# Normalizar los nombres de los artistas, canciones y álbumes en ambos datasets
grammy_merge['artist_clean'] = grammy_merge['artist'].apply(normalize_text)
grammy_merge['nominee_clean'] = grammy_merge['nominee'].apply(normalize_text)
spotify_merge['artists_clean'] = spotify_merge['artists'].apply(normalize_text)
spotify_merge['track_name_clean'] = spotify_merge['track_name'].apply(normalize_text)
spotify_merge['album_name_clean'] = spotify_merge['album_name'].apply(normalize_text)

# Exploding para que cada artista en una lista tenga su propia fila
spotify_merge = spotify_merge.explode('artists_clean')

# Merge directo basado en 'spotify_id' y 'track_id'
merged_df = pd.merge(grammy_merge, spotify_merge, left_on='spotify_id', right_on='track_id', how='left')

# Encontrar las filas sin match en el merge basado en spotify_id
no_match_df = merged_df[merged_df['track_id'].isnull()]

# Si hay filas que no tienen match, hacemos la coincidencia difusa
song_threshold = 70  # Umbral
artist_threshold = 70
album_threshold = 60

matches = []
for i, row in no_match_df.iterrows():
    song_match = process.extractOne(row['nominee_clean'], spotify_merge['track_name_clean'], scorer=fuzz.ratio)
    
    # Buscar coincidencia con track_name
    if song_match and song_match[1] >= song_threshold:
        matched_rows = spotify_merge[spotify_merge['track_name_clean'] == song_match[0]]
        artist_matches = []
        
        for _, matched_row in matched_rows.iterrows():
            artist_match = process.extractOne(row['artist_clean'], [matched_row['artists_clean']], scorer=fuzz.ratio)
            if artist_match and artist_match[1] >= artist_threshold:
                artist_matches.append(matched_row)  # Solo añadir la fila completa
            
        if artist_matches:
            matches.extend(artist_matches)
        else:
            matches.append(row)
    else:
        # Buscar coincidencia con album_name si no hay coincidencia en track_name
        album_match = process.extractOne(row['nominee_clean'], spotify_merge['album_name_clean'], scorer=fuzz.ratio)
        if album_match and album_match[1] >= album_threshold:
            matched_rows = spotify_merge[spotify_merge['album_name_clean'] == album_match[0]]
            artist_matches = []
            
            for _, matched_row in matched_rows.iterrows():
                artist_match = process.extractOne(row['artist_clean'], [matched_row['artists_clean']], scorer=fuzz.ratio)
                if artist_match and artist_match[1] >= artist_threshold:
                    artist_matches.append(matched_row)  # Solo añadir la fila completa
            
            if artist_matches:
                matches.extend(artist_matches)
            else:
                matches.append(row)
        else:
            matches.append(row)
    
    # Mostrar el progreso cada 100 filas
    if (i + 1) % 100 == 0:
        print(f"Procesadas {i + 1} filas de {len(no_match_df)}")

# Crear un DataFrame con las coincidencias
fuzzy_matched_df = pd.DataFrame(matches)

# Concatenar los resultados del merge directo y las coincidencias difusas
final_merged_df = pd.concat([merged_df.reset_index(drop=True), fuzzy_matched_df.reset_index(drop=True)], ignore_index=True)

# Eliminar columnas auxiliares excepto 'spotify_id'
final_merged_df = final_merged_df.drop(columns=['artist_clean', 'nominee_clean', 'artists_clean', 'track_name_clean', 'album_name_clean'], errors='ignore')

# Asegurarse de que la columna 'spotify_id' se mantenga
if 'spotify_id' not in final_merged_df.columns:
    print("La columna 'spotify_id' no está presente en el DataFrame final.")
else:
    print("La columna 'spotify_id' está presente en el DataFrame final.")

# Eliminar duplicados
final_merged_df = final_merged_df.drop_duplicates(subset=['nominee', 'artist', 'category'])

# Comprobar la cantidad de filas después del merge
merged_count = final_merged_df.shape[0]
print(f"Cantidad de filas después del merge: {merged_count}")

# Opcional: Verificar las primeras filas del DataFrame limpio
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
final_merged_df.head(20)

# Opcional: Guardar el DataFrame resultante en un archivo CSV
output_path = r'C:\Users\USUARIO\Documents\Workshop2\data\Merge1.csv'
final_merged_df.to_csv(output_path, index=False)
print(f"Merge completado y archivo guardado como '{output_path}'.")


We connect to the spotify api

In [136]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from dotenv import load_dotenv

# Cargar variables de entorno
load_dotenv()

# Configurar la autenticación
client_id = os.getenv('SPOTIFY_CLIENT_ID')
client_secret = os.getenv('SPOTIFY_CLIENT_SECRET')

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# Verifica que la autenticación haya sido exitosa
print("Conectado a Spotify API")


Conectado a Spotify API


We consume the spotify api to fill in null data like track and song characteristics

In [ ]:
import pandas as pd
import requests

# Ruta del archivo CSV
csv_path = r'C:\Users\USUARIO\Documents\Workshop2\data\Merge1.csv'

# Cargar el archivo CSV
final_merged_df = pd.read_csv(csv_path)

# Verificar si las columnas necesarias existen
required_columns = ['track_id', 'spotify_id']
for col in required_columns:
    if col not in final_merged_df.columns:
        raise ValueError(f"La columna '{col}' no se encuentra en el DataFrame.")

# Definir una función para consultar la API de Spotify (información del track)
def get_spotify_info(spotify_id, access_token):
    url = f"https://api.spotify.com/v1/tracks/{spotify_id}"
    headers = {
        "Authorization": f"Bearer {access_token}"
    }
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Lanza un error si la respuesta no es 200
        return response.json()  # Devuelve los datos de la canción
    except requests.exceptions.RequestException as e:
        print(f"Error al obtener información del track {spotify_id}: {e}")
        return None  # Manejo de error

# Definir una función para consultar la API de Spotify (características de audio)
def get_audio_features(spotify_id, access_token):
    url = f"https://api.spotify.com/v1/audio-features/{spotify_id}"
    headers = {
        "Authorization": f"Bearer {access_token}"
    }
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Lanza un error si la respuesta no es 200
        return response.json()  # Devuelve las características de audio
    except requests.exceptions.RequestException as e:
        print(f"Error al obtener características de audio para {spotify_id}: {e}")
        return None  # Manejo de error

# Token de acceso (debes actualizar esto con tu token válido)
ACCESS_TOKEN = 'BQAdZQgp9lRNQSjs-AzuyWQy85jEgZSIQPMmSahJCiIwpN-8owQ8mYx6Lpd0DeQPDgB_LteoMCUWngUimI1WeR94ezPj5pE2llwe_0eIAkXnJVzuE4Q'

# Iterar sobre el DataFrame para llenar los NaN
for index, row in final_merged_df[final_merged_df['track_id'].isnull()].iterrows():
    spotify_id = row['spotify_id']  # Asegúrate de que esta columna existe
    
    if pd.notna(spotify_id):  # Verificar si el spotify_id no es NaN
        try:
            # Obtener información del track de Spotify
            track_info = get_spotify_info(spotify_id, ACCESS_TOKEN)
            
            if track_info:
                # Rellenar columnas relevantes
                final_merged_df.loc[index, 'track_id'] = track_info.get('id')
                final_merged_df.loc[index, 'artists'] = ', '.join([artist['name'] for artist in track_info.get('artists', [])])
                final_merged_df.loc[index, 'album_name'] = track_info.get('album', {}).get('name')
                final_merged_df.loc[index, 'track_name'] = track_info.get('name')
                final_merged_df.loc[index, 'popularity'] = track_info.get('popularity')
                final_merged_df.loc[index, 'duration_ms'] = track_info.get('duration_ms')
            
                # Obtener características de audio
                audio_features = get_audio_features(spotify_id, ACCESS_TOKEN)
                
                if audio_features:
                    final_merged_df.loc[index, 'danceability'] = audio_features.get('danceability')
                    final_merged_df.loc[index, 'energy'] = audio_features.get('energy')
                    final_merged_df.loc[index, 'tempo'] = audio_features.get('tempo')
                    final_merged_df.loc[index, 'track_genre'] = track_info.get('genre', None)  # Si el género está disponible
                
            # Mostrar progreso cada 100 filas
            if index % 100 == 0:
                print(f"Información rellenada en fila {index}")
        
        except Exception as e:
            print(f"Error al procesar fila {index}: {e}")

# Comprobar cuántas filas siguen teniendo NaN después del proceso
remaining_nan_counts = final_merged_df.isnull().sum()
print("Conteo de NaN después del proceso:")
print(remaining_nan_counts)

# Opcional: Verificar las primeras filas del DataFrame limpio
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
print(final_merged_df.head(20))  # Asegúrate de imprimir el DataFrame

# Guardar el resultado actualizado en un nuevo archivo CSV (opcional)
#output_path = r'C:\Users\USUARIO\Documents\Workshop2\notebooks\Merged_Updated.csv'
#final_merged_df.to_csv(output_path, index=False)
#print(f"Archivo actualizado guardado en: {output_path}")


We check the columns and rows

In [149]:
# Opcional: Verificar las primeras filas del DataFrame limpio
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
final_merged_df.head(5)

,id,year,title,category,nominee,artist,workers,winner,spotify_id,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,danceability,energy,tempo,track_genre
0,1.0,2019.0,62nd Annual GRAMMY Awards (2019),Record Of The Year,Bad Guy,Billie Eilish,"Finneas O'Connell, producer; Rob Kinelski & Fi...",True,2Fxmhks0bxGSBdJ92vM42m,31018.0,2Fxmhks0bxGSBdJ92vM42m,Billie Eilish,"WHEN WE ALL FALL ASLEEP, WHERE DO WE GO?",bad guy,84.0,194087.0,0.701,0.425,135.128,electro
1,2.0,2019.0,62nd Annual GRAMMY Awards (2019),Record Of The Year,"Hey, Ma",Bon Iver,"BJ Burton, Brad Cook, Chris Messina & Justin V...",True,0RstfX9nRY1Lfuy1808MoT,NaN,0RstfX9nRY1Lfuy1808MoT,Bon Iver,"i,i","Hey, Ma",58.0,216706.0,0.325,0.286,144.296,None
2,3.0,2019.0,62nd Annual GRAMMY Awards (2019),Record Of The Year,7 rings,Ariana Grande,"Charles Anderson, Tommy Brown, Michael Foster ...",True,6ocbgoVGwYJhOv1GgI9NsF,20009.0,6ocbgoVGwYJhOv1GgI9NsF,Ariana Grande,"thank u, next",7 rings,84.0,178626.0,0.778,0.317,140.048,dance
3,4.0,2019.0,62nd Annual GRAMMY Awards (2019),Record Of The Year,Hard Place,H.E.R.,"Rodney “Darkchild” Jerkins, producer; Joseph H...",True,4IablJ6SqVNGY4vrseyKxu,NaN,4IablJ6SqVNGY4vrseyKxu,H.E.R.,Hard Place,Hard Place,54.0,271733.0,0.614,0.719,160.075,None
4,5.0,2019.0,62nd Annual GRAMMY Awards (2019),Record Of The Year,Talk,Khalid,"Disclosure & Denis Kosiak, producers; Ingmar C...",True,6g6A7qNhTfUgOSH7ROOxTD,NaN,6g6A7qNhTfUgOSH7ROOxTD,"Jason Derulo, 2 Chainz",Talk Dirty,Talk Dirty (feat. 2 Chainz),67.0,177685.0,0.760,0.652,100.315,None


We save the result in a csv to later upload it to a postgres database

In [ ]:
# Guardar el resultado actualizado en un nuevo archivo CSV (opcional)
output_path = r'C:\Users\USUARIO\Documents\Workshop2\notebooks\Merged_Updated.csv'
final_merged_df.to_csv(output_path, index=False)
print(f"Archivo actualizado guardado en: {output_path}")

Here we see how many columns with nulls there are, basically there are nulls because the token expired but that is fixed in the merge

In [143]:
# Excluir las columnas 'Unnamed: 0' y 'track_genre' al verificar NaN
columns_to_exclude = ['Unnamed: 0', 'track_genre']
filtered_df = final_merged_df.drop(columns=columns_to_exclude, errors='ignore')

# Ver el número total de filas y columnas en el DataFrame (sin excluir columnas)
total_filas, total_columnas = final_merged_df.shape
print(f"Total de filas: {total_filas}")
print(f"Total de columnas: {total_columnas}")

# Ver cuántas filas tienen NaN en alguna columna (excluyendo las columnas especificadas)
filas_con_nan = filtered_df.isna().any(axis=1).sum()
print(f"Filas con valores NaN (excluyendo 'Unnamed: 0' y 'track_genre'): {filas_con_nan}")

# Ver cuántas columnas tienen al menos un NaN (excluyendo las columnas especificadas)
columnas_con_nan = filtered_df.isna().any(axis=0).sum()
print(f"Columnas con valores NaN (excluyendo 'Unnamed: 0' y 'track_genre'): {columnas_con_nan}")


Total de filas: 4757
Total de columnas: 20
Filas con valores NaN (excluyendo 'Unnamed: 0' y 'track_genre'): 2222
Columnas con valores NaN (excluyendo 'Unnamed: 0' y 'track_genre'): 18


We see that the only nulls are in the characteristics of the song

In [145]:
# Excluir las columnas 'Unnamed: 0' y 'track_genre' al verificar NaN
columns_to_exclude = ['Unnamed: 0', 'track_genre','danceability','energy','tempo']
filtered_df = final_merged_df.drop(columns=columns_to_exclude, errors='ignore')

# Ver el número total de filas y columnas en el DataFrame (sin excluir columnas)
total_filas, total_columnas = final_merged_df.shape
print(f"Total de filas: {total_filas}")
print(f"Total de columnas: {total_columnas}")

# Ver cuántas filas tienen NaN en alguna columna (excluyendo las columnas especificadas)
filas_con_nan = filtered_df.isna().any(axis=1).sum()
print(f"Filas con valores NaN (excluyendo 'Unnamed: 0' ,'track_genre' ,'danceability','energy'y'tempo'): {filas_con_nan}")

# Ver cuántas columnas tienen al menos un NaN (excluyendo las columnas especificadas)
columnas_con_nan = filtered_df.isna().any(axis=0).sum()
print(f"Columnas con valores NaN (excluyendo 'Unnamed: 0' y 'track_genre','danceability','energy'y'tempo'): {columnas_con_nan}")


Total de filas: 4757
Total de columnas: 20
Filas con valores NaN (excluyendo 'Unnamed: 0' ,'track_genre' ,'danceability','energy'y'tempo'): 4
Columnas con valores NaN (excluyendo 'Unnamed: 0' y 'track_genre','danceability','energy'y'tempo'): 15
